In [1]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
import networkx as nx

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar,StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
g1 = nx.read_graphml("2019-12-31.graphml")
g2 = nx.read_graphml("2020-01-01.graphml")

In [3]:
G = nx.intersection(g1, g2)

In [4]:
subg1 = g1.subgraph(G.nodes())
subg2 = g2.subgraph(G.nodes())

In [5]:
subg1.number_of_nodes()

43417

In [6]:
subg2.number_of_nodes()

43417

In [7]:
nx.get_edge_attributes(subg1,'n_tx')

{('424638818', '50263989'): 1,
 ('424638818', '395698450'): 1,
 ('44906474', '454096221'): 2,
 ('445263208', '443088443'): 1,
 ('445263208', '423979401'): 3,
 ('445263208', '443928539'): 1,
 ('438347250', '23158888'): 1,
 ('438347250', '454626382'): 1,
 ('451811642', '423582443'): 2,
 ('451811642', '34515'): 1,
 ('451811642', '28479831'): 1,
 ('453913874', '454561140'): 1,
 ('447725134', '96780285'): 3,
 ('443914167', '454594238'): 1,
 ('311618383', '96780285'): 6,
 ('311618383', '33981853'): 1,
 ('46156554', '48917365'): 1,
 ('36695534', '28479831'): 1,
 ('52874346', '364047767'): 1,
 ('421458629', '96780285'): 3,
 ('421458629', '338655451'): 1,
 ('421458629', '348358470'): 1,
 ('421458629', '356823835'): 2,
 ('421458629', '433039918'): 1,
 ('40176048', '12309158'): 1,
 ('330691317', '12512653'): 1,
 ('330691317', '21934087'): 1,
 ('401026098', '340045788'): 1,
 ('439800629', '229000023'): 2,
 ('439800629', '43952585'): 1,
 ('439800629', '454129588'): 3,
 ('439800629', '52775162'): 1,

In [8]:
nx.set_edge_attributes(subg1, nx.get_edge_attributes(subg1, "n_tx"), name="weight")

In [9]:
nx.set_edge_attributes(subg2, nx.get_edge_attributes(subg2, "n_tx"), name="weight")

In [10]:
df_nodes1 = nx.to_pandas_adjacency(subg1)
df_nodes2 = nx.to_pandas_adjacency(subg2)

In [11]:
Gs1 = StellarGraph.from_networkx(
    subg1,node_features=df_nodes1
)
Gs2 = StellarGraph.from_networkx(
    subg2,node_features=df_nodes2
)

In [12]:
print(Gs1.info())

StellarDiGraph: Directed multigraph
 Nodes: 43417, Edges: 35200

 Node types:
  default: [43417]
    Features: float32 vector, length 43417
    Edge types: default-default->default

 Edge types:
    default-default->default: [35200]
        Weights: range=[1, 2513], mean=2.42068, std=21.0797
        Features: none


In [13]:
print(Gs2.info())

StellarDiGraph: Directed multigraph
 Nodes: 43417, Edges: 46496

 Node types:
  default: [43417]
    Features: float32 vector, length 43417
    Edge types: default-default->default

 Edge types:
    default-default->default: [46496]
        Weights: range=[1, 527], mean=1.70208, std=6.92294
        Features: none


In [14]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(Gs2)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=False
)

** Sampled 4649 positive and 4649 negative edges. **


In [15]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(Gs1)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=False
)

** Sampled 3520 positive and 3520 negative edges. **


In [16]:
batch_size = 20
epochs = 10
num_samples = [20, 10]

In [17]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [18]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [19]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [20]:
# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

/Users/liminghao/miniconda3/envs/stellar/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [21]:
prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [22]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

In [23]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

2023-05-09 16:44:46.357370: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


465/465 [==============================] - 657s 1s/step - loss: 3.8477 - acc: 0.5346

Train Set Metrics of the initial (untrained) model:
	loss: 4.4354
	acc: 0.5537

Test Set Metrics of the initial (untrained) model:
	loss: 3.8477
	acc: 0.5346


In [ ]:
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)

Epoch 1/20
352/352 - 1485s - loss: 0.7468 - acc: 0.6608 - val_loss: 1.3650 - val_acc: 0.5637 - 1485s/epoch - 4s/step
Epoch 2/20
352/352 - 1480s - loss: 0.5996 - acc: 0.6911 - val_loss: 1.3731 - val_acc: 0.5978 - 1480s/epoch - 4s/step
Epoch 3/20


In [ ]:
sg.utils.plot_history(history)

In [ ]:
train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))